In [18]:
import serial
import csv
import time
from datetime import datetime

def read_sensor_and_save_data(file_count, measurement_kind, pressure, measurement_method):
    file_format = "C:\\Users\\Mizuki\\University\\Reserch\\length_sensor\\JupyterLab\\data\\reflex\\reflex_{date}_{count}_{kind}_{pressure}_{method}.csv"
    #file_format = "C:\\Users\\Mizuki\\OneDrive - Kyoto University\\University\\Reserch\\length_sensor\\JupyterLab\\data\\real\\real_{date}_{count}_{kind}_{pressure}_{method}.csv"
    
    current_date = datetime.now().strftime("%Y%m%d")
    pressure_with_decimal = f"0.{pressure}"  
    csv_filename = file_format.format(date=current_date, count=file_count, kind=measurement_kind, pressure=pressure_with_decimal, method=measurement_method)
    
    try:
        ser = serial.Serial('COM6', 115200, timeout=1)
        
        # 最初のセンサー値取得時の時間を記録
        start_time = time.time()

        with open(csv_filename, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["Time", "Length", "Pressure", "Force"])

            while True:  # 無限ループ
                try:  
                    data = ser.readline().decode().strip()
                    current_time = time.time()
                    elapsed_time = int((current_time - start_time) * 1000)

                    print(f"{elapsed_time} - {data}")

                    if data and not data.startswith("Sensor reading stopped"):
                        parts = data.split(',')
                        if len(parts) == 3:
                            wire_length, pressure_val, force = parts
                            pressure_val = int(pressure_val) / 1000
                            writer.writerow([elapsed_time, wire_length, pressure_val, force])
                        else:
                            print("Invalid data format:", data)
                    elif data.startswith("Sensor reading stopped"):
                        print("Sensor reading stopped. Exiting loop.")
                        break
                except KeyboardInterrupt:
                    print("Interrupt received. Saving CSV file and exiting.")
                    break

    finally:
        ser.close()
        
measurement_method = input("測定方法を入力してください(d or s or p): ")
measurement_kind = input("測定するマッキベンの種類を入力してください(4s or 4l or 6 or s or u): ")

測定方法を入力してください(d or s or p):  reflex
測定するマッキベンの種類を入力してください(4s or 4l or 6 or s or u):  anta


In [2]:
pressure = input("圧力を入力してください(例:4): ")  

圧力を入力してください(例:4):  reaching


In [26]:
file_count = input("測定回数を入力してください(例:1): ") 
read_sensor_and_save_data(file_count, measurement_kind, pressure, measurement_method)

測定回数を入力してください(例:1):  8


1 - 163.80,16,0.00
2 - 163.80,16,0.00
2 - 163.80,16,0.00
2 - 163.90,16,0.00
2 - 163.90,16,0.00
6 - 163.90,16,0.00
6 - 163.90,16,0.00
6 - 163.90,16,0.00
6 - 163.90,15,0.00
28 - 163.90,18,0.00
45 - 163.90,16,0.00
65 - 163.90,16,0.00
87 - 163.90,16,0.00
106 - 164.00,16,0.00
126 - 164.00,16,0.00
147 - 164.00,16,0.00
167 - 164.00,16,0.00
187 - 164.00,16,0.00
200 - 164.00,16,0.00
224 - 164.00,16,0.00
239 - 164.00,16,0.00
264 - 164.10,16,0.00
274 - 164.10,16,0.00
304 - 164.10,16,0.00
321 - 164.10,16,0.00
337 - 164.10,16,0.00
354 - 164.10,16,0.00
383 - 164.20,16,0.00
403 - 164.20,16,0.00
418 - 164.20,16,0.00
447 - 164.20,16,0.00
467 - 164.20,16,0.00
486 - 164.20,16,0.00
506 - 164.20,16,0.00
520 - 164.30,16,0.00
536 - 164.30,16,0.00
552 - 164.30,16,0.00
583 - 164.30,16,0.00
601 - 164.30,16,0.00
620 - 164.40,16,0.00
647 - 164.40,16,0.00
665 - 164.40,16,0.00
684 - 164.40,18,0.00
704 - 164.40,16,0.00
720 - 164.50,16,0.00
745 - 164.50,16,0.00
753 - 164.50,16,0.00
787 - 164.50,18,0.00
800 - 164.50,1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter, MultipleLocator
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime


pressure_with_decimal = f"0.{pressure}"  

# データの読み込み
current_date = datetime.now().strftime("%Y%m%d")
input_file_path = f"C:\\Users\\Mizuki\\University\\Reserch\\length_sensor\\JupyterLab\\data\\reflex\\reflex_{current_date}_{file_count}_{measurement_kind}_{pressure_with_decimal}_{measurement_method}.csv"
df = pd.read_csv(input_file_path)


# "Length"列の重複を削除
cleaned_df = df[df['Length'] != df['Length'].shift()]

# クリーンなデータを別のファイルとして保存
output_file_path = f"C:\\Users\\Mizuki\\University\\Reserch\\length_sensor\\JupyterLab\\data\\reflex_processed\\reflex_processed_{current_date}_{file_count}_{measurement_kind}_{pressure_with_decimal}_{measurement_method}.csv"
cleaned_df.to_csv(output_file_path, index=False)

%matplotlib inline

current_date = datetime.now().strftime("%Y%m%d")
plt.rcParams['font.family'] = 'MS Gothic'  # 凡例やラベルに日本語を使っても文字化けしないようにする

# データの読み込み
data_path = f"C:\\Users\\Mizuki\\University\\Reserch\\length_sensor\\JupyterLab\\data\\reflex_processed\\reflex_processed_{current_date}_{file_count}_{measurement_kind}_{pressure_with_decimal}_{measurement_method}.csv"
dt = pd.read_csv(data_path)

# 列の選択
column = dt[["Pressure", "Length", "Force"]]

# 最初のPressureデータを取得して四捨五入
first_pressure = round(column["Pressure"].iloc[0], 2)  # 少数第三位を四捨五入

# グラフの作成
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

# 最初のデータからForceが最大値を取るデータまでの範囲（赤色）
ax.plot(column["Pressure"].iloc[:column["Force"].idxmax()+1], 
        column["Length"].iloc[:column["Force"].idxmax()+1], 
        column["Force"].iloc[:column["Force"].idxmax()+1], 
        c='r',marker = 'o',markersize = 2, label=f'{first_pressure}MPa - extension')

# Forceが最大値を取ったデータから最後のデータまでの範囲（青色）
ax.plot(column["Pressure"].iloc[column["Force"].idxmax()+1:], 
        column["Length"].iloc[column["Force"].idxmax()+1:], 
        column["Force"].iloc[column["Force"].idxmax()+1:], 
        c='b',marker = 'o',markersize = 2,label=f'{first_pressure}MPa - contraction')

# 軸ラベルの設定
ax.set_xlabel('Pressure(MPa)')
ax.set_ylabel('Length(mm)')
ax.set_zlabel('Force(N)')

# x軸のメモリフォーマットと設定
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))  # 少数第二位まで表示

# 凡例の表示
fig.legend(labels=['Extension', 'Contraction'], loc=(0.81,0.1), fontsize='large')

# グラフの保存
save_path = f'C:\\Users\\Mizuki\\University\\Reserch\\length_sensor\\JupyterLab\\graph\\reflex\\reflex_single_{current_date}_{file_count}_{measurement_kind}_{pressure_with_decimal}_{measurement_method}-1.jpg'
plt.savefig(save_path,bbox_inches='tight')

plt.tight_layout()  # 追加：グラフが適切にレイアウトされるように
plt.show()


#以下2D


# データの読み込み
#dt = pd.read_csv(f"C:\\Users\\Mizuki\\University\\Reserch\\length_sensor\\JupyterLab\\data\\real_processed\\p_real_{current_date}_{file_count}.csv")
dt = pd.read_csv(f"C:\\Users\\Mizuki\\University\\Reserch\\length_sensor\\JupyterLab\\data\\reflex_processed\\reflex_processed_{current_date}_{file_count}_{measurement_kind}_{pressure_with_decimal}_{measurement_method}.csv")

# 列の選択（Pressure列を削除）
column = dt[["Length", "Force"]]

# 最初のPressureデータを取得して四捨五入
first_pressure = round(dt["Pressure"].iloc[0], 2)  # 少数第三位を四捨五入

# 二次元プロット
fig, ax = plt.subplots(figsize=(10,6))

# Forceが最大値を取るまでの範囲を赤でプロット
ax.plot(column["Length"][:dt["Force"].idxmax() + 1], column["Force"][:dt["Force"].idxmax() + 1], c='r',marker = 'o',markersize = 2,label=f'{first_pressure}MPa - extension')

# Forceが最大値を取った後の範囲を青でプロット
ax.plot(column["Length"][dt["Force"].idxmax() + 1:], column["Force"][dt["Force"].idxmax() + 1:], c='b',marker = 'o',markersize = 2,label=f'{first_pressure}MPa - contraction')

# 軸ラベルの設定
ax.set_xlabel('Length (mm)')
ax.set_ylabel('Force (N)')

# 軸のメモリのフォーマットおよび設定
ax.yaxis.set_major_formatter(FormatStrFormatter('%.0f'))

# 凡例の表示
ax.legend(loc = 'upper left',fontsize = 'xx-large')


# 保存パスとファイル名を指定してグラフを画像として保存（JPG形式）
save_path = f'C:\\Users\\Mizuki\\University\\Reserch\\length_sensor\\JupyterLab\\graph\\reflex\\reflex_{current_date}_{file_count}_{measurement_kind}_{pressure_with_decimal}_{measurement_method}-2.jpg'
plt.savefig(save_path,bbox_inches='tight')

# グラフの表示
plt.subplots_adjust(top=0.9, bottom=0.1, left=0.1, right=0.9, hspace=0.4, wspace=0.4)
plt.tight_layout()# 追加：グラフが適切にレイアウトされるように
plt.show()
